# <font color = 'indianred'>**Experiements Machine Translation checkpoints** </font>

In [ ]:
!nvidia-smi

Wed Nov  1 00:12:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set Up Environment

In [ ]:
from pathlib import Path
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount("/content/drive")
    !pip install datasets transformers evaluate wandb accelerate seqeval transformers[sentencepiece] sacrebleu bert_score -U -qq
    base_folder = Path("/content/drive/MyDrive")


from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, pipeline
from datasets import load_dataset, DatasetDict, Dataset, ClassLabel, Sequence
import evaluate
from evaluate import evaluator


import wandb
import torch
import gc
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import textwrap

data_folder = base_folder/'datasets/'
model_folder = base_folder/'models/machine-translation'
model_folder.mkdir(exist_ok=True,parents=True)
data_folder.mkdir(exist_ok=True,parents=True)

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**---------DO NOT CHANGE ANY OF THE FUNCTIONS------------**

## Function to Load Dataset

In [ ]:
def load_custom_dataset(dataset_name,lang1, lang2):
    from datasets import load_dataset
    dataset = load_dataset(dataset_name,lang1=lang1,lang2= lang2)
    # dataset = dataset.select_columns(select_column_name)
    # dataset = dataset.rename_columns({text_column_name: 'text', label_column_name : 'labels'})

    # selected_columns = {
    # 'text': dataset['train'][text_column_name],
    # 'label': dataset['train'][label_columns_name],
    # }

    # # Create a new dataset with the selected columns
    # dataset_selected_columns = Dataset.from_dict(selected_columns)
    # dataset_selected_columns.set_format(type='pandas')
    # df = dataset_selected_columns[:]
    # df['text'] = df['text'].fillna('')
    # dataset_selected_columns.reset_format()

    # if class_names:
    #     dataset_selected_columns = dataset_selected_columns.cast_column('label', ClassLabel(names = class_names))

    return dataset['train']

## Function to Split Dataset

In [ ]:
def split_dataset(dataset, train_size, val_size, test_size):
    test_val_splits = dataset.train_test_split(train_size=train_size, seed=42)
    train_split= test_val_splits['train']
    test_size_new =test_size/(test_size + val_size)
    test_val_splits = test_val_splits['test'].train_test_split(test_size=test_size_new, seed=42)
    val_split = test_val_splits['train']
    test_split = test_val_splits['test']

    train_val_dataset = DatasetDict({'train': train_split, 'val': val_split})
    test_dataset = DatasetDict({'test': test_split})

    return train_val_dataset, test_dataset

## Function to Create smaller subset

In [ ]:
def get_small_balanced_subset(train_val_dataset, test_dataset, train_size, val_size, test_size):
    train_split_small = train_val_dataset['train'].shuffle(seed=42).select(range(train_size))
    val_split_small = train_val_dataset['val'].shuffle(seed=42).select(range(val_size))
    test_split_small = test_dataset['test'].shuffle(seed=42).select(range(test_size))

    # combine train, val splits into one dataset
    train_val_subset = DatasetDict({'train': train_split_small, 'val': val_split_small})

    # create test dataset from test split
    test_subset= DatasetDict({'test': test_split_small})

    return train_val_subset, test_subset


## Function for Tokenization

In [ ]:
def get_tokenized_dataset(checkpoint, dataset):


    def tokenize_fn(batch):
        tokenizer = AutoTokenizer.from_pretrained(checkpoint,return_tensor= 'pt')
        max_length = 128
        inputs = [example['en'] for example in batch['translation']]
        targets = [example['fr'] for example in batch['translation']]
        model_inputs = tokenizer(text = inputs, text_target=targets, max_length=max_length, truncation=True)
        return model_inputs

    tokenized_dataset = dataset.map(tokenize_fn,
                              batched = True,
                              remove_columns=dataset['train'].column_names)
    # tokenized_dataset.set_format(type="torch")
    return tokenized_dataset


## Function to Create Datasets

In [ ]:
def setup_dataset():

    # Constants for loading and splitting

    dataset_name = 'kde4'
    lang1='en'
    lang2='fr'

    # 1. Load Dataset
    dataset = load_custom_dataset(dataset_name = dataset_name, lang1=lang1, lang2=lang2)

    # 2. Split Dataset
    train_val_dataset, test_dataset = split_dataset(dataset,0.6,0.2,0.2)

    # 3. Get Small Balanced Subset
    train_val_subset, test_subset = get_small_balanced_subset(train_val_dataset, test_dataset ,3000 ,800,800)

    return train_val_subset, test_subset

## Function to Initialize Model

In [ ]:
from transformers import GenerationConfig
def initialize_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint)
    # generation_config = GenerationConfig.from_pretrained(checkpoint)
    # id2label = {}
    # for id_, label_ in enumerate(class_names):
    #     id2label[str(id_)] = label_

    # label2id = {}
    # for id_, label_ in enumerate(class_names):
    #     label2id[label_] = id_

    # config.id2label = id2label
    # config.label2id = label2id

    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, config=config)
    return model

## Function to Compute Metrics

In [ ]:
bleu_metric = evaluate.load("sacrebleu")
bert_metric = evaluate.load('bertscore')

def compute_metrics(preds_and_labels):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint,return_tensor= 'pt')
    preds, labels = preds_and_labels

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_cleaned = [pred.strip() for pred in decoded_preds]
    decoded_labels_cleaned = [label.strip() for label in decoded_labels]

    bleu_score = bleu_metric.compute(predictions=decoded_preds_cleaned, references=decoded_labels_cleaned)
    # bert_score = bert_metric.compute(predictions=decoded_preds_cleaned, references=decoded_labels_cleaned,model_type='microsoft/deberta-xlarge-mnli')
    bert_score = bert_metric.compute(predictions=decoded_preds_cleaned, references=decoded_labels_cleaned,lang='en')

    return {"bleu_score" : bleu_score['score'], 'bert_score' : np.mean(bert_score['f1'])}

## Function to set Trainer

In [ ]:
def get_trainer(model, training_args, train_dataset, eval_dataset, compute_metrics, tokenizer, data_collator):
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator= data_collator
    )
    return trainer

## Function to free memory

In [ ]:
import gc
def free_memory():
    """
    Attempts to free up memory by deleting variables and running Python's garbage collector.
    """
    gc.collect()
    for device_id in range(torch.cuda.device_count()):
        torch.cuda.set_device(device_id)
        torch.cuda.empty_cache()
    gc.collect()


## Function to tokenize dataset and, train and eval models

<font color = 'indianred'>**ALLOWED TO CHANGE THE BLOCK IN THE FUNCTION BELOW**

In [ ]:
def tokenize_train_evaluate_log(training_args, checkpoint, base_folder,
                                 train_val_subset, compute_metrics):
    # 1. Free memory
    free_memory()

    # 2. Setup wandb
    wandb.login()
    %env WANDB_PROJECT = nlp_course_translation

    ######################## ALLOWED TO CHANGE THIS BLOCK ################################################

    # MAKE SURE THE BASE FOLDER IS SETUP CORRECTLY
    #  YOU CAN CHANGE THIS LINE IF YOU WANT TO SAVE IN A DIFFERENT FOLDER

    model_folder = base_folder/'models/translation'
    model_folder.mkdir(exist_ok=True, parents=True)

    ######################## ALLOWED TO CHANGE THIS BLOCK ################################################


    # 3. Get Tokenized Dataset and Data Collator
    train_val_tokenized_dataset = get_tokenized_dataset(checkpoint, train_val_subset)

    # 4. Initialize Model and Tokenizer
    model = initialize_model(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model)
    trainer = get_trainer(model, training_args, train_val_tokenized_dataset['train'],
                          train_val_tokenized_dataset['val'], compute_metrics, tokenizer, data_collator)

    # 6. Train and Evaluate
    trainer.train()
    trainer.evaluate(train_val_tokenized_dataset['val'])

    # 7. Log Metrics and Plot
    # log_and_plot_confusion_matrix(trainer, train_val_tokenized_dataset['val'], class_names)

    best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
    wandb.log({"best_model_checkpoint_step": best_model_checkpoint_step})
    print(f"The best model was saved at step {best_model_checkpoint_step}.")

    wandb.finish()

## Initial Training Arguments

DO NOT CHANGE ANY ARGUMENTS

In [ ]:
from transformers import Seq2SeqTrainingArguments
from transformers import GenerationConfig
check = 'Helsinki-NLP/opus-mt-en-fr'
generation_config = GenerationConfig.from_pretrained(check)
# Configure training parameters
training_args = Seq2SeqTrainingArguments(
    # Training-specific configurations
    num_train_epochs=3,  # Total number of training epochs
    # weight_decay=0.01,  # Apply L2 regularization to prevent overfitting
    learning_rate=2e-5,  # Step size for the optimizer during training
    lr_scheduler_type= "linear",
    # Optimizer
    optim="adamw_torch",  # Optimizer,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon = 1e-8,
    # warmup_steps=10,
    predict_with_generate=True,
    generation_config= generation_config,
    # memory and speed related arguments
    # Number of samples per training batch for each device
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,  # Number of samples per eval batch for each device

    gradient_checkpointing=True,  # memory
    # fp16 = True, # Speed
    # bf16=True,
    # tf32=True, # speed
    # evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy="steps",  # Evaluate model at specified step intervals
    eval_steps=10,  # Perform evaluation every 10 training steps
    # Checkpoint settings
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=20,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=1,  # Retain only the best and the most recent model checkpoints
    metric_for_best_model="bleu_score",
    greater_is_better= True,
    # Experiment logging configurations (commented out in this example)
    logging_strategy="steps",
    logging_steps=10,
    report_to="wandb",  # Log metrics and results to Weights & Biases platform
    # Experiment name for Weights & Biases
    run_name="translation-exp1",
)

# Experiments

## Dataset hyperparameters

In [ ]:
train_val_subset, test_subset = setup_dataset()

Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

## Experiment 1 : Helsinki-NLP/opus-mt-en-fr

### Trainer hyperparameters

In [ ]:
checkpoint = 'Helsinki-NLP/opus-mt-en-fr'
# generation_config = checkpoint
# training_args_dict = training_args.to_dict() # Convert TrainingArguments to dictionary
training_args.run_name = f'{checkpoint}-{1000}' # Update the run_name
# new_training_args = Seq2SeqTrainingArguments(**training_args_dict)

In [ ]:
tokenize_train_evaluate_log(training_args= training_args, checkpoint=checkpoint, base_folder=base_folder,
                              train_val_subset=train_val_subset,
                             compute_metrics=compute_metrics)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


env: WANDB_PROJECT=nlp_course_HW6-Part-D


Step,Training Loss,Validation Loss,Bleu Score,Bert Score
10,1.857300,1.624620,39.316051,0.918084
20,1.634700,1.577800,40.535493,0.919212
30,1.538100,1.548185,40.651708,0.920015
40,1.473500,1.530189,40.578417,0.919253
50,1.495900,1.518404,40.599892,0.918360
60,1.253400,1.509264,41.334516,0.918893
70,1.250600,1.502716,41.195639,0.919042
80,1.182000,1.498448,41.114712,0.918900
90,1.323900,1.495667,41.188114,0.918445


The best model was saved at step 60.


eval/bert_score,▆█▁▃▄▃▁▂▃▂▂▂
eval/bleu_score,▆█▁▅▅▅▅▇▇▆▇▇
eval/loss,▃▁█▆▄▃▂▂▂▁▁▂
eval/runtime,██▁▂▁▁▂▂▂▂▂▂
eval/samples_per_second,▂▃█▄▆▅▂▃▂▂▁▃
eval/steps_per_second,▁▁█▆▇▇▆▆▆▆▅▆
train/epoch,▁▁▁▁▂▁▁▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇████
train/global_step,▁▁▂▂▃▁▁▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇█████
train/learning_rate,██▇███▇▆▅▄▃▃▂▁
train/loss,▇▅▅▇██▆▅▄▄▂▂▁▂
train/total_flos,▁


## Experiment 2 : huggingface-course/marian-finetuned-kde4-en-to-fr

### Trainer hyperparameters

In [ ]:
checkpoint =  "huggingface-course/marian-finetuned-kde4-en-to-fr"
# generation_config = checkpoint
# training_args_dict = training_args.to_dict() # Convert TrainingArguments to dictionary
training_args.run_name = f'{checkpoint}-{1000}' # Update the run_name
# new_training_args = Seq2SeqTrainingArguments(**training_args_dict)


In [ ]:
tokenize_train_evaluate_log(training_args= training_args, checkpoint=checkpoint, base_folder=base_folder,
                             train_val_subset=train_val_subset,
                             compute_metrics=compute_metrics)

env: WANDB_PROJECT=nlp_course_HW6-Part-D


Step,Training Loss,Validation Loss,Bleu Score,Bert Score
10,0.876700,0.744852,53.009238,0.940068
20,0.765400,0.748105,53.134935,0.940771
30,0.799500,0.750667,53.417505,0.940538
40,0.771800,0.752638,53.042947,0.940481
50,0.736400,0.754588,52.768482,0.939717
60,0.618200,0.755068,52.861922,0.940041
70,0.626900,0.755633,53.000034,0.940515
80,0.567900,0.756211,52.843544,0.940535
90,0.655700,0.756405,52.875642,0.940531


The best model was saved at step 30.


eval/bert_score,▃█▆▆▁▃▆▆▆▆
eval/bleu_score,▄▅█▄▁▂▃▂▂█
eval/loss,▁▃▅▆▇▇███▅
eval/runtime,▁▂▆█▆▆▆▆▆▆
eval/samples_per_second,█▇▃▁▃▃▃▃▂▃
eval/steps_per_second,█▇▃▁▃▃▃▃▂▃
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇████
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇█████
train/learning_rate,█▇▆▅▄▄▃▂▁
train/loss,█▅▆▆▅▂▂▁▃
train/total_flos,▁


## Experiment 3 : huggingface-course/marian-finetuned-kde4-en-to-fr

### Trainer hyperparameters

In [ ]:
checkpoint =  "huggingface-course/marian-finetuned-kde4-en-to-fr"
# generation_config = checkpoint
# training_args_dict = training_args.to_dict() # Convert TrainingArguments to dictionary
training_args.run_name = f'{checkpoint}-{3000}' # Update the run_name
# new_training_args = Seq2SeqTrainingArguments(**training_args_dict)


In [ ]:
tokenize_train_evaluate_log(training_args= training_args, checkpoint=checkpoint, base_folder=base_folder,
                             train_val_subset=train_val_subset,
                             compute_metrics=compute_metrics)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=nlp_course_HW6-Part-D


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

wandb: Currently logged in as: akshat_patil. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Bleu Score,Bert Score
10,0.800000,0.720003,54.447100,0.939571
20,0.789600,0.722916,54.501809,0.940024
30,0.740000,0.725105,54.382099,0.939970
40,0.907300,0.726169,53.443088,0.940027
50,0.808700,0.726191,53.223986,0.939469
60,0.875400,0.726607,52.826663,0.939355
70,0.732600,0.727691,54.369138,0.939927
80,0.924300,0.728140,54.328762,0.939975
90,0.916900,0.728452,53.987538,0.939691
100,0.798800,0.728292,54.440599,0.939356


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The best model was saved at step 20.


eval/bert_score,▂▅▄▅▂▁▄▄▃▁▃▅█▇▃▃▅▆▅▅▃▁▃▃▃▄▄▄▅
eval/bleu_score,██▇▃▃▁▇▇▆█▇▇█▇▆▆▆▆▆▆▇▆▇▇▆▆▆▆█
eval/loss,▁▂▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇███████▂
eval/runtime,█▁▂▂▃▃▅▁▃▂▄▁▄▃▃▂▅▁▄▂▄▃▃▂▃▁▄▁▃
eval/samples_per_second,▁█▇▆▆▆▄▇▅▇▄█▅▆▆▆▄█▅▇▅▆▆▇▅█▅▇▆
eval/steps_per_second,▁█▇▇▆▆▄█▆▇▅█▅▆▆▆▄█▅▇▅▆▆▇▅█▅█▆
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,▅▅▄▇▅▇▄██▅█▃▃▃▄▄▃▃▃▁▄▂▃▂▂▄▂▃
train/total_flos,▁


# Conclusion
In conclusion, we conducted a series of experiments to evaluate the translation performance of different models for English to French translation. Specifically, we used the following model:

huggingface-course/marian-finetuned-kde4-en-to-fr: This model is a fine-tuned version of Helsinki-NLP/opus-mt-en-fr on the kde4 dataset, with the following training parameters:
Learning rate: 2e-05
Optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
Learning rate scheduler type: Linear
The results of these experiments are as follows:

* Experiment 1: We used the baseline model Helsinki-NLP/opus-mt-en-fr, which achieved an evaluation BLEU score of 41.33452 when trained on 1000 samples.

* Experiment 2: We employed the huggingface-course/marian-finetuned-kde4-en-to-fr model, training it on 1000 samples. This model outperformed the baseline, achieving an evaluation BLEU score of 53.4175.

* Experiment 3: We continued to use the huggingface-course/marian-finetuned-kde4-en-to-fr model, but this time, we trained it on 3000 samples. The performance further improved, resulting in an evaluation BLEU score of 54.50181.

It's evident that fine-tuning the huggingface-course/marian-finetuned-kde4-en-to-fr model on the kde4 dataset led to significant improvements in translation quality compared to the baseline Helsinki-NLP/opus-mt-en-fr model. Additionally, increasing the training data to 3000 samples resulted in a further enhancement in translation performance. The choice of optimization settings, including the learning rate and optimizer, played a crucial role in achieving these improvements.